محمدپیام تائبی
40104867

بخش اول

In [20]:
%%writefile matrix_mult_pipe.c

#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h>
#include <sys/time.h>

int** matrix1;
int** matrix2;
int** matrix3;
int** result;

int thread_size;
int m, k, n;

typedef struct {
    int row;
    int col;
} Index;

int pipefd[2];

void* write_to_pipe(void* arg) {
    for (int i = 0; i < m; i++) {
        for (int j = 0; j < n; j++) {
            Index index;
            index.row = i;
            index.col = j;
            write(pipefd[1], &index, sizeof(Index));
        }
    }
}

void* worker_thread(void* arg) {
    Index index;
    read(pipefd[0], &index, sizeof(Index));
    printf("Thread %ld: Reading index <%d, %d>\n", pthread_self(), index.row, index.col);

    int row = index.row;
    int col = index.col;
    int sum = 0;
    for (int i = 0; i < k; i++)
        sum += matrix1[row][i] * matrix2[i][col];

    result[row][col] = sum;
    return NULL;
}

void normal_matrix_multiplication(int** A, int** B, int** C, int m, int k, int n) {
    for (int i = 0; i < m; i++) {
        for (int j = 0; j < n; j++) {
            int value = 0;
            for (int l = 0; l < k; l++)
                value += A[i][l] * B[l][j];
            C[i][j] = value;
        }
    }
}

int** allocate_matrix(int row, int col) {
    int** A = (int**)malloc(sizeof(int*) * row);
    for (int i = 0; i < row; i++)
        A[i] = (int*)malloc(sizeof(int) * col);
    return A;
}

void initialize_matrix(int** A, int row, int col) {
    printf("Enter matrix elements for a %d x %d matrix:\n", row, col);
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            printf("Enter element at position (%d, %d): ", i, j);
            scanf("%d", &A[i][j]);
        }
    }
}

void print_matrix(int** A, int row, int col, const char* label) {
    printf("%s:\n", label);
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            printf("%d\t", A[i][j]);
        }
        printf("\n");
    }
}

int main() {
    srand(time(0));

    printf("Enter values for m, k, n, and number of threads (e.g., 2 2 2 4): ");
    scanf("%d %d %d %d", &m, &k, &n, &thread_size);

    matrix1 = allocate_matrix(m, k);
    matrix2 = allocate_matrix(k, n);
    matrix3 = allocate_matrix(m, n);
    result = allocate_matrix(m, n);

    printf("--------------------\n");
    printf("Matrix sizes: m=%d, k=%d, n=%d\n", m, k, n);
    printf("Thread size is: %d\n", thread_size);

    initialize_matrix(matrix1, m, k);
    initialize_matrix(matrix2, k, n);

    print_matrix(matrix1, m, k, "Input Matrix 1");
    print_matrix(matrix2, k, n, "Input Matrix 2");

    pthread_t* threads = (pthread_t*)malloc(sizeof(pthread_t) * (thread_size + 1));

    // pipe initialization handling
    if (pipe(pipefd) == -1) {
        perror("pipe");
        exit(1);
    }

    struct timeval start, end;
    gettimeofday(&start, NULL);

    pthread_create(&threads[thread_size], NULL, write_to_pipe, NULL);
    for (int i = 0; i < thread_size; i++) {
        pthread_create(&threads[i], NULL, worker_thread, NULL);
    }

    for (int i = 0; i < thread_size; i++) {
        pthread_join(threads[i], NULL);
    }

    gettimeofday(&end, NULL);
    double threaded_execution_time = (end.tv_sec - start.tv_sec) + (end.tv_usec - start.tv_usec) / 1000.0;

    printf("Multi-threaded Multiplication Execution Time:\t %.6f ms\n", threaded_execution_time);

    gettimeofday(&start, NULL);
    normal_matrix_multiplication(matrix1, matrix2, matrix3, m, k, n);
    gettimeofday(&end, NULL);
    double normal_execution_time = (end.tv_sec - start.tv_sec) + (end.tv_usec - start.tv_usec) / 1000.0;
    if (normal_execution_time < 0)
        normal_execution_time *= -1.0;
    printf("Normal Multiplication Execution Time: \t\t %.6f ms \n", normal_execution_time);

    print_matrix(matrix3, m, n, "Output Matrix (Multi-threaded)");
    print_matrix(result, m, n, "Output Matrix (Normal)");

    // close(pipefd[0]);
    // close(pipefd[1]);

    return 0;
}


Overwriting matrix_mult_pipe.c


In [21]:
!gcc -o matrix_mult_pipe matrix_mult_pipe.c -lpthread
!./matrix_mult_pipe

Enter values for m, k, n, and number of threads (e.g., 2 2 2 4): 2 2 2 4
--------------------
Matrix sizes: m=2, k=2, n=2
Thread size is: 4
Enter matrix elements for a 2 x 2 matrix:
Enter element at position (0, 0): 1
Enter element at position (0, 1): 2
Enter element at position (1, 0): 3
Enter element at position (1, 1): 4
Enter matrix elements for a 2 x 2 matrix:
Enter element at position (0, 0): 1
Enter element at position (0, 1): 1
Enter element at position (1, 0): 1
1
Enter element at position (1, 1): Input Matrix 1:
1	2	
3	4	
Input Matrix 2:
1	1	
1	1	
Thread 133247434716736: Reading index <0, 0>
Thread 133247426324032: Reading index <0, 1>
Thread 133247417931328: Reading index <1, 0>
Thread 133247409538624: Reading index <1, 1>
Multi-threaded Multiplication Execution Time:	 0.306000 ms
Normal Multiplication Execution Time: 		 0.001000 ms 
Output Matrix (Multi-threaded):
3	3	
7	7	
Output Matrix (Normal):
3	3	
7	7	


بخش دوم

In [12]:
%%writefile mult.c


#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h>
#include <sys/time.h>

int** matrix1;
int** matrix2;
int** matrix3;
int** result;

int thread_size;
int m, k, n, iterations;

typedef struct {
    int row;
    int col;
} Index;

int pipefd[2];

void* write_to_pipe(void* arg) {
    for (int i = 0; i < m; i++) {
        for (int j = 0; j < n; j++) {
            Index index;
            index.row = i;
            index.col = j;
            write(pipefd[1], &index, sizeof(Index));
        }
    }
}

void* worker_thread(void* arg) {
    Index index;
    read(pipefd[0], &index, sizeof(Index));
    //printf("Thread %ld: Reading index <%d, %d>\n", pthread_self(), index.row, index.col);

    int row = index.row;
    int col = index.col;
    int sum = 0;
    for (int i = 0; i < k; i++)
        sum += matrix1[row][i] * matrix2[i][col];

    result[row][col] = sum;
    return NULL;
}

void normal_matrix_multiplication(int** A, int** B, int** C, int m, int k, int n) {
    for (int i = 0; i < m; i++) {
        for (int j = 0; j < n; j++) {
            int value = 0;
            for (int l = 0; l < k; l++)
                value += A[i][l] * B[l][j];
            C[i][j] = value;
        }
    }
}

int** allocate_matrix(int row, int col) {
    int** A = (int**)malloc(sizeof(int*) * row);
    for (int i = 0; i < row; i++)
        A[i] = (int*)malloc(sizeof(int) * col);
    return A;
}

void initialize_matrix(int** A, int row, int col) {
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++)
            A[i][j] = rand() % RAND_MAX;
    }
}

int main() {
    srand(time(0));

    printf("Enter values for m, k, n, and number of iterations (e.g., 1000 512 512 8): ");
    scanf("%d %d %d %d", &m, &k, &n, &iterations);

    matrix1 = allocate_matrix(m, k);
    matrix2 = allocate_matrix(k, n);
    matrix3 = allocate_matrix(m, n);
    result = allocate_matrix(m, n);

    for (int iter = 0; iter < iterations; iter++) {
        printf("--------------------\nIteration %d\n", iter + 1);
        printf("Matrix sizes: m=%d, k=%d, n=%d\n", m, k, n);

        thread_size = 2 << iter;
        printf("Thread size is: %d\n", thread_size);

        initialize_matrix(matrix1, m, k);
        initialize_matrix(matrix2, k, n);
        initialize_matrix(matrix3, m, n);

        pthread_t* threads = (pthread_t*)malloc(sizeof(pthread_t) * (thread_size + 1));

        // pipe initialization handling
        if (pipe(pipefd) == -1) {
            perror("pipe");
            exit(1);
        }

        struct timeval start, end;
        gettimeofday(&start, NULL);

        pthread_create(&threads[thread_size], NULL, write_to_pipe, NULL);
        for (int i = 0; i < thread_size; i++) {
            pthread_create(&threads[i], NULL, worker_thread, NULL);
        }

        for (int i = 0; i < thread_size; i++) {
            pthread_join(threads[i], NULL);
        }

        gettimeofday(&end, NULL);
        double threaded_execution_time = (end.tv_sec - start.tv_sec) + (end.tv_usec - start.tv_usec) / 1000.0;

        printf("Multi-threaded Multiplication Execution Time:\t %.6f ms\n", threaded_execution_time);

        gettimeofday(&start, NULL);
        normal_matrix_multiplication(matrix1, matrix2, matrix3, m, k, n);
        gettimeofday(&end, NULL);
        double normal_execution_time = (end.tv_sec - start.tv_sec) + (end.tv_usec - start.tv_usec) / 1000.0;
        if (normal_execution_time < 0)
            normal_execution_time *= -1.0;
        printf("Normal Multiplication Execution Time: \t\t %.6f ms \n", normal_execution_time);

        // close(pipefd[0]);
        // close(pipefd[1]);
    }

    return 0;
}


Overwriting mult.c


In [13]:
!gcc -o mult mult.c -lpthread
!./mult

Enter values for m, k, n, and number of iterations (e.g., 1000 512 512 8): 1000 512 512 10
--------------------
Iteration 1
Matrix sizes: m=1000, k=512, n=512
Thread size is: 2
Multi-threaded Multiplication Execution Time:	 0.275000 ms
Normal Multiplication Execution Time: 		 172.718000 ms 
--------------------
Iteration 2
Matrix sizes: m=1000, k=512, n=512
Thread size is: 4
Multi-threaded Multiplication Execution Time:	 0.379000 ms
Normal Multiplication Execution Time: 		 178.474000 ms 
--------------------
Iteration 3
Matrix sizes: m=1000, k=512, n=512
Thread size is: 8
Multi-threaded Multiplication Execution Time:	 0.616000 ms
Normal Multiplication Execution Time: 		 167.904000 ms 
--------------------
Iteration 4
Matrix sizes: m=1000, k=512, n=512
Thread size is: 16
Multi-threaded Multiplication Execution Time:	 1.830000 ms
Normal Multiplication Execution Time: 		 662.335000 ms 
--------------------
Iteration 5
Matrix sizes: m=1000, k=512, n=512
Thread size is: 32
Multi-threaded Mu